IMPORTAMOS LIBRERIAS

In [28]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.optim as optim


CARGAMOS IMAGENES

In [29]:
##!pip install kaggle

In [30]:
#from kaggle.api.kaggle_api_extended import KaggleApi

#api = KaggleApi()
#api.authenticate()

# Replace "xhlulu/140k-real-and-fake-faces" with your dataset name and specify the download path
#api.dataset_download_files("xhlulu/140k-real-and-fake-faces", path="your_download_path", unzip=True)


DIVIDIMOS

In [31]:
import os
import shutil

# Define las rutas donde quieres almacenar las imágenes de entrenamiento y prueba
train_path = 'dataset/train'
test_path = 'dataset/test'


# Asegúrate de que las carpetas 'real' y 'fake' existan dentro de 'train' y 'test'
os.makedirs(os.path.join(train_path, 'real2'), exist_ok=True)
os.makedirs(os.path.join(train_path, 'fake2'), exist_ok=True)
os.makedirs(os.path.join(test_path, 'real'), exist_ok=True)
os.makedirs(os.path.join(test_path, 'fake'), exist_ok=True)

DEFINIMOS TRANSFORMACIONES

In [32]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


CARGAMOS DATOS AL SISTEMA

In [33]:
train_data = datasets.ImageFolder(root='dataset/train', transform=transform)
test_data = datasets.ImageFolder(root='dataset/test', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


CARGAMOS ALEXNET PREENTRENADO

In [34]:
model = models.alexnet(pretrained=True)
model.classifier[6] = nn.Linear(4096, 2)  # Cambia la última capa para 2 clases
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')


DEFINIMOS CRITERIO DE PERDIDA Y OPTIMIZADOR

- criterio

Compara la salida del modelo con la etiqueta real y calcula qué tan lejos está la predicción del valor verdadero.
Penaliza errores asignando un valor más alto a los errores grandes y un valor más bajo a los errores menores, generando un "pérdida" que el modelo debe minimizar.

- optimizador 

Combina los beneficios de los algoritmos de momento y adaptación de tasa de aprendizaje. Esto permite un entrenamiento más estable y eficiente.
Ajusta los parámetros de forma dinámica basándose en el valor de la pérdida, ayudando a encontrar los valores óptimos rápidamente y evitando quedar atascado en mínimos locales.

In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


ENTRENAMIENTO DEL MODELO

device = 'cuda' if torch.cuda.is_available() else 'cpu'

se utiliza para determinar si el entrenamiento del modelo debe realizarse en una GPU (unidad de procesamiento gráfico) o en una CPU (unidad de procesamiento central). Vamos a desglosar esto:

In [36]:
def train_model(model, criterion, optimizer, train_loader, test_loader, epochs=5):
    best_accuracy = 0.0  
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.train()

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Evaluación en cada epoch
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')
        test_accuracy = evaluate_model(model, test_loader)
        print(f'Test Accuracy: {test_accuracy:.2f}%')

        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            best_model_wts = model.state_dict()
        
def evaluate_model(model, test_loader):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.eval()  # Cambia a modo evaluación
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    model.train()  # Cambia de nuevo a modo entrenamiento
    return 100 * correct / total


ENTRENAR Y EVALUAR

In [37]:
train_model(model, criterion, optimizer, train_loader, test_loader, epochs=1)

Epoch 1/1, Loss: 0.7134746017928322
Test Accuracy: 50.00%


In [ ]:
train_model(model, criterion, optimizer, train_loader, test_loader, epochs=5)

REALIZAR PREDICCIONES

In [ ]:
from PIL import Image

def predict_image(image_path, model):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Añadir batch dimension
    image = image.to('cuda' if torch.cuda.is_available() else 'cpu')
    
    model.eval()
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    return 'IA' if predicted.item() == 0 else 'Real'
